<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/ClubRaiders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

1. Welcome to Club Raiders.
2. 'The Club' or 'The Cabal' is a mysterious group in the game Elite: Dangerous.
3. The Club is apparently related to two other long-standing mysteries in Elite: Dangerous - Raxxla and The Dark Wheel.
4. Almost everything we know about The Club is conjecture.
5. A group in Elite Dangerous known as The Children of Raxxla has provided reliable information about other in-game mysteries in the past.
6. The Children of Raxxla have released documents about The Club.  This information may or may not be reliable.
7. Two of these documents are known as The Holdstock Report and The Club Unmasked.
8. These two documents suggest that some in-game minor factions support The Club, and others oppose them.

This document considers the factions identified on the chart below, in particular where these factions are operating throughout the Milky Way.

![Big Crazy Club Diagram](https://github.com/HausReport/ClubRaiders/blob/master/info/My%20Club%20Diagram.png?raw=true)

Helping individual players and groups find Club factions operating in their vicinity is part one of the Bolan Plan: **Identify, Engage, Eradicate**.


# Number Crunching

In [1]:
#@title
import sys
sys.executable
sys.path

%load_ext google.colab.data_table
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"

from dateutil.relativedelta import relativedelta
import datetime
from datetime import timezone 

!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

#import cufflinks as cf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px



  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-jlwvl274/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-jlwvl274/craid
  Created wheel for craid: filename=craid-0.47-cp36-none-any.whl size=11671 sha256=bcc9f5b5ff7e1e494a1c72dca731ba157b5187ed9d2f39539c1a9185a7fae67a
  Stored in directory: /tmp/pip-ephem-wheel-cache-_7ikn3ny/wheels/59/74/32/b75ca6dbfddc83071fd4a4a492a26c1066d06840be16c7d535
Successfully built craid
  Found existing installation: craid 0.46
    Uninstalling craid-0.46:
      Successfully uninstalled craid-0.46
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [0]:
#@title 
import craid.Club
import craid.LoadDataFromEDDB
from craid.eddb.FactionInstance import FactionInstance
import craid.Crunch
craid.LoadDataFromEDDB.load_data()
ret = craid.Crunch.getSystemsArray()
club_systems_arr: FactionInstance = ret[0]

df = craid.Crunch.getDataFrame(club_systems_arr)
#
# Dataframe of all club factions except Emp Grace
#
filter = df[ ~df[ 'factionName' ].str.contains("Emperor's Grace") ]



# Intercepted Message

In [3]:
#@title 


frame = df

dt = datetime.datetime.now() 
utc_time = dt.replace(tzinfo = timezone.utc) 

new_date = utc_time + relativedelta(years=1)
print("TO: CMDR [REDACTED]/OPS")
print("FM: CMDR E[REDACTED]/MAIA DATA")
print("DT: " + new_date.strftime("%d-%b-%Y %H:%M"))
print("RE: HAXDAT/CLUB STUFF TO SHOOT")
print("===================================")

print()
print("Hey.  Spent the last two nights up to my ears in data from the last breach.  Here are the ")
print("highlights.  I think they're pretty compelling, but I'm juiced to the gills to stay awake.")

print()

print( "Club factions are currently active in " + str(frame['systemName'].count()) +" systems, home to " + '{:,}'.format(int(frame['population'].sum(axis=0))) + " souls." )
idx = frame['x'].idxmin()

print('They span the galaxy from {0} in {1}, {2} light years west of Sol'.format(frame.iloc[idx,1], frame.iloc[idx,0], abs(int(frame.iloc[idx,2]))))
idx = frame['x'].idxmax()

print('to {0} in {1}, {2} light years to the east.'.format(frame.iloc[idx,1], frame.iloc[idx,0], abs(int(frame.iloc[idx,2]))))

print()

print("It's encouraging to note that club factions only control {0} of these systems, or about {1:.0%}.".format(frame['control'].sum(), frame['control'].sum()/frame['systemName'].count()))
print("Their influence in all systems they operate in ranges from {0:.0%} to {1:.0%}, averaging {2:.0%}, but".format(df['influence'].min()/100.0, df['influence'].max()/100.0, df['influence'].mean()/100.0))

uncontrolled = frame[ ~frame[ 'control' ]]

print("in systems they don't control, these numbers fall to {0:.0%}-{1:.0%} influence with an average of only {2:.0%}.".format(uncontrolled['influence'].min()/100.0, uncontrolled['influence'].max()/100.0, uncontrolled['influence'].mean()/100.0))

gaxx =uncontrolled['influence'].describe()

print("In these non-control systems, half of their factions have less than {0:.0%} influence and a full\nquarter are operating at less than {1:.0%}.".format(gaxx[5]/100,gaxx[4]/100))

print()
print("As you know, we forced a number of club retreats lately.  Here's today's hot list:")
print()

gg = filter.copy()
gg = gg[gg['influence']<6]
gg = gg[gg['influence']>0]
gg = gg[~gg['isHomeSystem']]
gg=  gg.sort_values('influence')
print(gg.to_string(columns=['factionName','systemName','influence','vulnerable']))


print()
print("In other words: there's some low-hanging fruit to pick out there.")


print()
print("You said you just wanted something to shoot at.  Go shoot 'em.  I should have the new system")
print("online in a few days.  Let me know if you want another update after we xxxxxxx the new terminal")
print("tonight.  In the meantime, I'm gonna cause some trouble out here in the Pleiades.")

print()
print("Fly dangerous,")
print("-E")

TO: CMDR [REDACTED]/OPS
FM: CMDR E[REDACTED]/MAIA DATA
DT: 23-May-2021 08:10
RE: HAXDAT/CLUB STUFF TO SHOOT

Hey.  Spent the last two nights up to my ears in data from the last breach.  Here are the 
highlights.  I think they're pretty compelling, but I'm juiced to the gills to stay awake.

Club factions are currently active in 712 systems, home to 335,697,755,020 souls.
They span the galaxy from Sirius Corporation in Sothis, 352 light years west of Sol
to Gallant Investment Brokers in Exphiay, 283 light years to the east.

It's encouraging to note that club factions only control 145 of these systems, or about 20%.
Their influence in all systems they operate in ranges from 0% to 100%, averaging 20%, but
in systems they don't control, these numbers fall to 0%-40% influence with an average of only 12%.
In these non-control systems, half of their factions have less than 11% influence and a full
quarter are operating at less than 8%.

As you know, we forced a number of club retreats lately

# Systems with Club Influence by Allegiance

The graph below shows systems where Club factions are operating.  The ball of blue systems is largely Emperor's Grace factions that may or may not have much to do with Club operations.

In [0]:
#import plotly.express as px
#fig = px.scatter_3d(df, x='x', y='z', z='y',color='influence', hover_data={  'systemName', 'factionName'})
#fig.show()

If we remove the Emperor's Grace factions for a moment, it will give a clearer picture of the other factions in play.

In [0]:
#figx = px.scatter_3d(filter, x='x', y='z', z='y',color='allegiance', hover_data={  'systemName', 'factionName'})
#figx.show()

The table below presents the same data in a tabular format.  


In [6]:
@interact_manual(maxInfluence=(0, 100.0, 0.1))
def show_factions_with(IncludeHomeSystems=False, IncludeControlSystems=True, IncludeEmperorsGrace=False, maxInfluence=12.0):
    return df.loc[(~df['isHomeSystem']|IncludeHomeSystems) & (~df['control']|IncludeControlSystems) & ( ~df[ 'factionName' ].str.contains("Emperor's Grace")|IncludeEmperorsGrace ) & (df['influence'] <= maxInfluence)]

interactive(children=(Checkbox(value=False, description='IncludeHomeSystems'), Checkbox(value=True, descriptio…

# More crunching

In [7]:
#@title 


res = [i + " in " + j for i, j in zip(filter['factionName'], filter['systemName'])] 
#%run TwoDimensionalRegionsPlot.py

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=filter['x'],
    y=filter['y'],
    mode="markers",  hovertext= res ))

fig.add_trace(go.Scatter(
    x=[-270, -270,  -95,  140,   -20,   -75,   70,   275,   30],
    y=[-200, -50,   150,  120,     0,  -175,   80,   -80,   120],
    mode="text",
    name="Regions",
    text=["Turner Reach", "Sirius Reach", "Corporate Way","Sirius East","Sirius Core","Pleiades", "The Old Worlds", "Gallant Beach", "Bentonia"],
    textposition="bottom center"
))

# Add shapes
fig.update_layout(
    shapes=[
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-350,
            y0=-175,
            x1= -175,
            y1= -250,
            opacity=0.2,
            fillcolor="blue",
            line_color="blue",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-375,
            y0=-100,
            x1= -125,
            y1= 50,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-110,
            y0=75,
            x1= -75,
            y1= 130,
            opacity=0.2,
            fillcolor="green",
            line_color="green",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=120,
            y0=-20,
            x1= 160,
            y1= 100,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-40,
            y0= -30,
            x1= 50,
            y1= 30,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-100,
            y0= -175,
            x1= -50,
            y1= -75,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
       dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  60,
            y0= 10,
            x1= 80,
            y1= 60,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  250,
            y0= -150,
            x1= 300,
            y1= -100,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  25,
            y0= 135,
            x1= 40,
            y1= 175,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
    ],
)
fig.update_layout(showlegend=False)




In [8]:
res = [i + " in " + j for i, j in zip(filter['factionName'], filter['systemName'])] 
#%run TwoDimensionalRegionsPlot.py

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=filter['x'],
    y=filter['z'],
    mode="markers",  hovertext= res ))

fig.add_trace(go.Scatter(
    x=[-270, -270,  -95,  140,   -20,   -75,   70,   275,   30],
    y=[-200, -50,   150,  120,     0,  -175,   80,   -80,   120],
    mode="text",
    name="Regions",
    text=["Turner Reach", "Sirius Reach", "Corporate Way","Sirius East","Sirius Core","Pleiades", "The Old Worlds", "Gallant Beach", "Bentonia"],
    textposition="bottom center"
))

# Add shapes
fig.update_layout(
    shapes=[
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-350,
            y0=-175,
            x1= -175,
            y1= -250,
            opacity=0.2,
            fillcolor="blue",
            line_color="blue",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-375,
            y0=-100,
            x1= -125,
            y1= 50,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-110,
            y0=75,
            x1= -75,
            y1= 130,
            opacity=0.2,
            fillcolor="green",
            line_color="green",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=120,
            y0=-80,
            x1= 160,
            y1= 130,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-40,
            y0= -30,
            x1= 50,
            y1= 30,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-100,
            y0= -400,
            x1= -50,
            y1= -200,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
       dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  60,
            y0= 10,
            x1= 80,
            y1= 60,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  250,
            y0= -40,
            x1= 300,
            y1= -0,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  25,
            y0= 135,
            x1= 40,
            y1= 175,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
    ],
)
fig.update_layout(showlegend=False)

# Galactic Areas of Club Influence

Again, setting aside the Emperor's Grace factions for clarity, there are around 10 general areas where Club factions tend to operate.

In [9]:
fig.show()

# All Club Factions as Comma-Delimited Text

If you like to work with data, you might find the comma-delimited text below useful:


In [10]:
#@title 
for xcs in club_systems_arr:
    cs: FactionInstance = xcs
    #if( "Emperor's Grace" in cs.get_name() ): continue
    #vulnerable = cs.isVulnerable()
    #if (vulnerable == False): continue
    cs.printCSV()

Vodyanes Emperor's Grace,27 G. Caeli,75.84,-62.09,-33.34,Empire,18.00,,23-May-2020 02:18
Worster Insurance,64 Ceti,-37.06,-100.06,-79.12,Independent,3.40,Low Influence,22-May-2020 20:42
Abroin Universal PLC,Abroin,-94.53,110.09,-40.50,Independent,42.00,,20-May-2020 02:22
HIP 5183 Emperor's Grace,Achuar,26.09,-231.44,5.94,Empire,12.30,,20-May-2020 16:55
Rishair Emperor's Grace,Adachit,107.28,-163.84,44.31,Empire,8.31,,21-May-2020 02:30
Vodyanes Emperor's Grace,Aha Wa,90.38,-56.62,-17.56,Empire,9.70,,23-May-2020 01:55
Umaspi Emperor's Grace,Ahti,4.06,-79.09,-7.72,Empire,9.10,,22-May-2020 16:45
Ekono Emperor's Grace,Aiga,66.62,-147.56,102.78,Empire,16.40,,22-May-2020 17:19
HIP 101031 Emperor's Grace,Ailurii,56.38,-125.28,157.06,Empire,8.80,,14-Mar-2020 00:50
Hodack Prison Colony,Akandi,62.03,10.75,38.88,Independent,4.00,,23-May-2020 01:43
Bill Turner,Alioth,-33.66,72.47,-20.66,Alliance,0.00,,23-May-2020 01:15
Turner Research Group,Alioth,-33.66,72.47,-20.66,Alliance,6.11,,23-May-2020 01:1

# Experiments

In [11]:
combo = widgets.Combobox(
    # value='John',
    placeholder='Choose Someone',
    options=['Paul', 'John', 'George', 'Geremy'],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)
output = widgets.Output()

display(combo, output)

def on_value_change(change):
        print(change['new'])
combo.observe(on_value_change, names='value')

Combobox(value='', description='Combobox:', ensure_option=True, options=('Paul', 'John', 'George', 'Geremy'), …

Output()

In [12]:
df.describe()

,x,y,z,population,influence
count,712.000000,712.000000,712.000000,7.120000e+02,712.000000
mean,50.087166,-99.345725,-6.712430,4.714856e+08,19.603731
std,86.562344,82.879015,137.738573,2.133469e+09,18.117462
min,-352.781250,-263.656250,-952.718750,0.000000e+00,0.000000
25%,8.460938,-155.703125,-41.773438,3.085150e+04,8.200000
50%,68.578125,-116.468750,23.437500,2.538760e+05,12.400000
75%,110.179688,-50.507812,72.593750,6.404160e+06,21.200000
max,283.843750,175.000000,168.343750,2.278087e+10,100.000000


In [13]:
a = -69.0625
b = 22.37
c = -148.09375
#a=-80.15625
#b=144.09375
#c=-333.375
df.query(f'sqrt( (x-{a})**2 + (y-{b})**2 + (z-{c})**2)<100')

,systemName,factionName,x,y,z,allegiance,isHomeSystem,population,influence,updated,control,vulnerable
54,BD+33 801,Sirius Hot2Cold,-31.31250,-25.03125,-107.87500,Independent,False,35800,14.0859,2020-05-22 22:05:58,False,
101,Chicoana,Sirius Hot2Cold,-35.00000,-23.21875,-110.50000,Independent,False,266162,4.6000,2020-05-22 18:35:22,False,
118,Daik,Sirius Hyperspace,-69.06250,-22.37500,-148.09375,Independent,False,2200,68.4369,2020-05-17 23:56:22,True,
172,Hemaki,Sirius Hot2Cold,-38.87500,-22.56250,-115.50000,Independent,False,2362890899,7.3926,2020-05-17 05:30:47,False,
417,Mooramba,Sirius Hyperspace,-68.62500,-17.50000,-149.12500,Independent,False,2752,46.5000,2020-05-22 22:32:37,True,
441,Ngalia,Sirius Hyperspace,-74.62500,-35.03125,-165.96875,Independent,True,9695033,6.1061,2020-05-22 19:26:56,False,
503,Ross 591,Sirius Hot2Cold,-32.18750,-28.09375,-111.37500,Independent,True,4100889,11.7882,2020-05-22 03:26:22,False,Civil War
686,HIP 17412,Sirius Hyperspace,-71.25000,-41.59375,-167.62500,Independent,False,734001,3.3000,2020-05-13 14:49:04,False,Low Influence
691,HIP 17655,Sirius Hyperspace,-83.40625,-32.90625,-178.37500,Independent,False,57282,17.9000,2020-05-20 20:23:04,False,
702,Wolf 202,Sirius Hyperspace,-70.15625,-27.59375,-140.21875,Independent,False,18776,7.2072,2020-05-20 01:09:20,False,Anarchy


In [14]:
a = -69.0625
b = 22.37
c = -148.09375
dist = 150
tit = f'Club systems within {dist}ly of ({a},{b},{b})'
#a=-80.15625
#b=144.09375
#c=-333.375
fie = df.query(f'sqrt( (x-{a})**2 + (y-{b})**2 + (z-{c})**2)<{dist}')
figz = px.scatter_3d(fie, x='x', y='z', z='y',color='influence', hover_data={  'systemName', 'factionName'},title=tit)
figz.show()

In [15]:
dd = [("Fred",(10,10,10)), ("Joe", (20,-10,15)) ]

def f(x):
    return x

interact(f, x=dd);


interactive(children=(Dropdown(description='x', options=(('Fred', (10, 10, 10)), ('Joe', (20, -10, 15))), valu…

In [0]:
pfs = ret[1]

In [24]:
combo = widgets.Combobox(
    # value='John',
    placeholder='Choose a Player Faction',
    options= list(pfs.keys()), #nameArray, #[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)], #[('1',1),('2',2),('3',3)], #pfs,
    description='Faction:',
    ensure_option=True,
    disabled=False
)
output = widgets.Output()

display(combo, output)

def on_value_change(change):
        print(pfs[change['new']])
combo.observe(on_value_change, names='value')

Combobox(value='', description='Faction:', ensure_option=True, options=('*Pyrite Delta League', '*Sky Line', '…

Output()

(106.71875, -16.78125, 37.21875)
(-64.65625, -148.9375, -330.40625)


In [0]:
sss = ret[2]

In [23]:
combo = widgets.Combobox(
    # value='John',
    placeholder='Choose Inhabited System',
    options= list(sss.keys()), #nameArray, #[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)], #[('1',1),('2',2),('3',3)], #pfs,
    description='System:',
    ensure_option=True,
    disabled=False
)
output = widgets.Output()

display(combo, output)

def on_value_change(change):
        print(sss[change['new']])
combo.observe(on_value_change, names='value')

Combobox(value='', description='Inhabited System:', ensure_option=True, options=('1 G. Caeli', '1 Geminorum', …

Output()

(75.75, 48.75, 70.75)
(-25.375, 1.125, -20.84375)


In [28]:
combo = widgets.Combobox(
    # value='John',
    placeholder='Choose Inhabited System',
    options= list(sss.keys()), #nameArray, #[('Galileo', 0), ('Brahe', 1), ('Hubble', 2)], #[('1',1),('2',2),('3',3)], #pfs,
    description='System:',
    ensure_option=True,
    disabled=False
)
output = widgets.Output()

display(combo, output)



def on_value_change(change):
  sysName = change['new']
  sys = sss[sysName]
  a = sys[0]
  b = sys[1]
  c = sys[2]
  dist = 150
  tit = f'Club systems within {dist}ly of {sysName}'
  fie = df.query(f'sqrt( (x-{a})**2 + (y-{b})**2 + (z-{c})**2)<{dist}')
  figz = px.scatter_3d(fie, x='x', y='z', z='y',color='influence', hover_data={  'systemName', 'factionName'},title=tit)
  figz.show()

combo.observe(on_value_change, names='value') 

Combobox(value='', description='System:', ensure_option=True, options=('1 G. Caeli', '1 Geminorum', '1 Hydrae'…

Output()